<a href="https://colab.research.google.com/github/frdprd/examples/blob/master/Finviz_DashboardList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install finvizfinance

In [4]:
import pandas as pd
from finvizfinance.quote import finvizfinance
from finvizfinance.screener.overview import Overview
from finvizfinance.screener.financial import Financial
from finvizfinance.screener.valuation import Valuation

In [ ]:
foverview = Overview()
filter_1 = {'Country':'USA', 'Market Cap.': '+Mid (over $2bln)'} #'Index':'S&P 500', 
foverview.set_filter(filters_dict=filter_1)
df = foverview.screener_view()
df.count()

In [ ]:
#download fundamentals with same filter to get ROE
ffinancial = Financial()
ffinancial.set_filter(filters_dict=filter_1)
df2 = ffinancial.screener_view()
df2.count()

In [ ]:
#download valuation ratios with same filter
fvaluation = Valuation()
fvaluation.set_filter(filters_dict=filter_1)
df3 = fvaluation.screener_view()
df3.count()

In [ ]:
#Then, join
df = pd.merge(df, df2, how='left', on=['Ticker'])
df = pd.merge(df, df3, how='left', on=['Ticker'])
df.count()

In [10]:
#keep 1500 largest cap
df=df.nlargest(1500, ['Market Cap'])
#drop rows for real estate
df = df[df.Sector != 'Real Estate']
#keep only useful columns
df4 = df[['Ticker','Company','Sector','Industry','ROE','P/E_y','P/S','P/FCF']]
df4.count()

Ticker      1402
Company     1402
Sector      1402
Industry    1402
ROE         1312
P/E_y       1114
P/S         1381
P/FCF        959
dtype: int64

In [19]:
#calculate medians by industry
#rename columns valuation ratios
df4 = df4.rename(columns={
    'P/E_y': 'pe',
    'P/S': 'ps',
    'P/FCF': 'pfcf'})
industry_medians = df4.groupby(['Industry']).agg({'pe':'median', 
                                                            'ps':'median', 
                                                            'pfcf':'median'})
#rename columns median
industry_medians = industry_medians.rename(columns={
    'pe': 'ind_med_pe',
    'ps': 'ind_med_ps',
    'pfcf': 'ind_med_pfcf'})
#industry_medians['Industry'] = industry_medians.index.get_level_values(level=0)
df5 = pd.merge(df4,industry_medians, how='left', on=['Industry'])
df5.count()

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Ticker          1402
Company         1402
Sector          1402
Industry        1402
ROE             1312
pe              1114
ps              1381
pfcf             959
ind_med_pe      1390
ind_med_ps      1392
ind_med_pfcf    1383
dtype: int64

In [21]:
#apply filters based on medians
df6 = df5[(df5.pe < df5.ind_med_pe) & (df5.ps < df5.ind_med_ps) & (df5.pfcf < df5.ind_med_pfcf)]
df6.count()

Ticker          194
Company         194
Sector          194
Industry        194
ROE             193
pe              194
ps              194
pfcf            194
ind_med_pe      194
ind_med_ps      194
ind_med_pfcf    194
dtype: int64

In [27]:
#keep the top 10 ROE by sector
#sorting first
df6 = df6.sort_values(['Sector', 'ROE'], ascending=False)
df7=df6.groupby('Sector').head(10)

# alternative latest Pandas version???
#df7 = df6.groupby('Industry')['ROE'].nlargest(10).nlargest(80, keep='first')
df7

,Ticker,Company,Sector,Industry,ROE,pe,ps,pfcf,ind_med_pe,ind_med_ps,ind_med_pfcf
656,UGI,UGI Corporation,Utilities,Utilities - Regulated Gas,0.243,5.55,0.75,34.36,20.120,1.870,80.815
240,ED,"Consolidated Edison, Inc.",Utilities,Utilities - Regulated Electric,0.08,18.92,2.08,13.75,19.990,2.310,14.110
1159,VNT,Vontier Corporation,Technology,Scientific & Technical Instruments,1.002,6.12,0.96,12.07,27.570,3.220,32.280
94,AMAT,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,0.556,11.58,2.82,17.53,16.820,3.490,18.490
274,ON,ON Semiconductor Corporation,Technology,Semiconductors,0.361,16.80,3.45,19.93,17.810,4.230,21.480
...,...,...,...,...,...,...,...,...,...,...,...
478,WLK,Westlake Corporation,Basic Materials,Specialty Chemicals,0.342,4.45,0.80,5.75,23.340,1.440,18.280
219,DOW,Dow Inc.,Basic Materials,Chemicals,0.306,6.17,0.55,4.47,7.810,0.945,18.465
800,HUN,Huntsman Corporation,Basic Materials,Chemicals,0.303,4.70,0.57,6.56,7.810,0.945,18.465
359,MOS,The Mosaic Company,Basic Materials,Agricultural Inputs,0.293,6.10,1.08,11.67,14.295,2.040,38.305


In [25]:
#keeping the top80 ROE
df7 = df7.sort_values(['ROE'], ascending=False)
df8=df7.head(80).sort_values(['Sector'], ascending=False).reset_index(drop=True)
df8


,Ticker,Company,Sector,Industry,ROE,pe,ps,pfcf,ind_med_pe,ind_med_ps,ind_med_pfcf
0,UGI,UGI Corporation,Utilities,Utilities - Regulated Gas,0.243,5.55,0.75,34.36,20.120,1.870,80.815
1,AMKR,"Amkor Technology, Inc.",Technology,Semiconductors,0.232,7.11,0.72,19.92,17.810,4.230,21.480
2,VNT,Vontier Corporation,Technology,Scientific & Technical Instruments,1.002,6.12,0.96,12.07,27.570,3.220,32.280
3,ON,ON Semiconductor Corporation,Technology,Semiconductors,0.361,16.80,3.45,19.93,17.810,4.230,21.480
4,AMAT,"Applied Materials, Inc.",Technology,Semiconductor Equipment & Materials,0.556,11.58,2.82,17.53,16.820,3.490,18.490
...,...,...,...,...,...,...,...,...,...,...,...
75,LYB,LyondellBasell Industries N.V.,Basic Materials,Specialty Chemicals,0.448,5.01,0.48,8.82,23.340,1.440,18.280
76,DOW,Dow Inc.,Basic Materials,Chemicals,0.306,6.17,0.55,4.47,7.810,0.945,18.465
77,CC,The Chemours Company,Basic Materials,Specialty Chemicals,0.791,5.40,0.63,12.78,23.340,1.440,18.280
78,OLN,Olin Corporation,Basic Materials,Specialty Chemicals,0.575,5.50,0.74,4.42,23.340,1.440,18.280
